In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import RocCurveDisplay, classification_report, roc_curve
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from xgboost import XGBClassifier

from tools_ml.data_process import filter_features
from tools_ml.models import get_pipeline

In [2]:
df_raw = pd.read_csv("data/application_train.csv")

# Dropando a coluna de gênero e SK_ID_CURR
df_raw.drop("CODE_GENDER", inplace=True, axis=1)
df_raw.drop("SK_ID_CURR", inplace=True, axis=1)

# Filtrando as features com mais de 50% de valores nulos
df_raw_fil = filter_features(df_raw, 0.5)

FileNotFoundError: [Errno 2] No such file or directory: 'dados/application_train.csv'

In [ ]:
# Features Numéricas
features_num = df_raw_fil.select_dtypes(include=np.number).columns.tolist()
features_num.remove('TARGET')

# Features Categóricas
features_cat = df_raw_fil.select_dtypes(exclude=np.number).columns.tolist()

# Selecionando features que possuem menos de 10 categorias
df_num = df_raw_fil[features_num]
novas_cat = df_num.nunique()[df_num.nunique()<10].sort_values()
novas_cat.drop([
    'DEF_30_CNT_SOCIAL_CIRCLE',
    'DEF_60_CNT_SOCIAL_CIRCLE',
    'AMT_REQ_CREDIT_BUREAU_HOUR',
    'AMT_REQ_CREDIT_BUREAU_DAY',
    'AMT_REQ_CREDIT_BUREAU_WEEK'
], inplace=True)

features_cat = features_cat + novas_cat.index.tolist()
features_num = df_raw_fil.columns.drop(features_cat).tolist()
features_num.remove('TARGET')

In [ ]:
# Separando em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    df_raw_fil.drop(columns='TARGET'),
    df_raw_fil['TARGET'],
    train_size=0.7,
    random_state=42,
    stratify=df_raw_fil['TARGET']
)

In [ ]:
# Proporção entre 0 e 1 na variável alvo
ratio = y_train.value_counts()[0]/y_train.value_counts()[1]

pipe_final = get_pipeline(
    features_cat=features_cat,
    features_num=features_num,
    classifier=XGBClassifier(random_state=42, scale_pos_weight=ratio)
)

In [ ]:
# Hyperparam tuning
espaco_hiper = {
    'classifier__learning_rate' : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    'classifier__max_depth' : [3, 4, 5, 6, 8, 10, 12, 15],
    'classifier__min_child_weight' : [1, 3, 5, 7 ],
    'classifier__gamma': [0.0, 0.1, 0.2 , 0.3, 0.4],
    'classifier__colsample_bytree' : [0.3, 0.4, 0.5 , 0.7]
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rand_xgb = RandomizedSearchCV(
    estimator=pipe_final,
    param_distributions=espaco_hiper,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-2,
    verbose=5,
    random_state=42
)

rand_xgb.fit(X_train, y_train)

In [ ]:
# Getting predictions
y_pred = rand_xgb.predict(X_test)

In [ ]:
# Model report
print(classification_report(y_test, y_pred))

In [ ]:
df_report = classification_report(y_test, y_pred, output_dict=True)
df_report = pd.DataFrame(df_report).transpose()
df_report

In [ ]:
# Params and Score
print("Best Parameters:")
for param, value in rand_xgb.best_params_.items():
    print(f"  - {param.title()}: {value:.2g}")
print(f"Best Score: {rand_xgb.best_score_:.2g}") 

In [ ]:
# Roc Curve
fpr, tpr, thr = roc_curve(y_test, y_pred)
RocCurveDisplay(fpr = fpr, tpr = tpr).plot()
plt.plot([0,1], [0,1], '--', color = 'y', linewidth= 1)
plt.show()